In [110]:
import numpy as np
import matplotlib as mp
import time
import random

In [111]:
# parameters for genetic algorithm
max_weight = 10
population_size = 15
mutation_probability = 0.02
crossover_probability = 0.8
generations = 10

In [112]:
# loading all the points from the data files into arrays

tset_file = open("data/training_set_v2", "r")
tlabel_file = open("data/training_labels_v2", "r")
vset_file = open("data/validation_set_v2", "r")
vlabel_file = open("data/validation_labels_v2", "r")

tset = []
tlabel = []
vset = []
vlabel = []

for i in tset_file.readlines():
    x0,x1,x2 = i.split(sep=',')
    tset += [[float(x0),float(x1),float(x2)]]

tset_file.close()


for i in tlabel_file.readlines():
    di = i.split()
    tlabel += [float(di[0])]

tlabel_file.close()

for i in vset_file.readlines():
    x0,x1,x2 = i.split(sep=',')
    vset += [[float(x0),float(x1),float(x2)]]

vset_file.close()


for i in vlabel_file.readlines():
    di = i.split()
    vlabel += [float(di[0])]

vlabel_file.close()

In [113]:
#ao ∈ [0, 2], a1 ∈ [−2, 0], and a2 ∈ [−1, 1].
def my_fun(x,y,a0,a1,a2):
    return (a0*np.cbrt(x-5)) + a1*np.cbrt(y+5) + a2

def obj_function(pred, actVal):
    #computing the mean square error
    return np.square(np.subtract(pred, actVal)).mean() 

def gen_population(size):
    population = []
    for _ in range(size):
        population.append([random.randrange(0,3), random.randrange(-2,1), random.randrange(-1,2)])
    
    print("Generated population of size ", size)
    return population

def evaluate(chromosome):
    predictions = []
    
    for i in range(len(tlabel)):
        predictions.append(my_fun(tset[i][0],tset[i][0],chromosome[0], chromosome[1], chromosome[2]))
    
    fitness = obj_function(predictions, tlabel)
    return -fitness
pop = gen_population(8)

Generated population of size  8


In [114]:
def roulette_wheel_selection(population):
    fitness_values = [evaluate(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)

    # Calculate selection probabilities
    selection_probs = [fitness / total_fitness for fitness in fitness_values]

    # Perform roulette wheel spin
    selected_index = -1
    spin = random.uniform(0, 1)
    cumulative_prob = 0.0

    for i, prob in enumerate(selection_probs):
        cumulative_prob += prob
        if spin <= cumulative_prob:
            selected_index = i
            break

    return population[selected_index]
def select_parents(population):
    parents = []
    #generate parent pairs array the half the size of the population
    for _ in range(len(population)//2):
        parents.append([roulette_wheel_selection(population),roulette_wheel_selection(population)])
    return parents
parent_pairs = select_parents(pop)

In [117]:

# Function to perform uniform crossover between two parents
def uniform_crossover(parent1, parent2):
    # Create an empty child chromosome
    child1 = [None]*3
    child2 = [None]*3

    # Uniformly select genes from parents to create the child chromosome
    for _ in range(len(parent1)):
        r = random.uniform(0,1)
        child1 = parent1 * r + parent2*(1-r)
        child2 = parent1 * (1-r) + parent2 * r
    print(f"Crossover beetween {parent1} and {parent2} Resulted in {child1} and {child2}")
    #print(f"Resulted in {child1} and {child2}\n")
    return child1, child2

for i in parent_pairs:
    uniform_crossover(np.array(i[0]),np.array(i[1]))


Crossover beetween [ 0 -2  0] and [ 1 -2  0] Resulted in [ 0.90497936 -2.          0.        ] and [ 0.09502064 -2.          0.        ]
Crossover beetween [ 2 -2  1] and [ 1 -2  1] Resulted in [ 1.54355941 -2.          1.        ] and [ 1.45644059 -2.          1.        ]
Crossover beetween [ 1 -2  0] and [1 0 1] Resulted in [ 1.         -1.76328773  0.11835614] and [ 1.         -0.23671227  0.88164386]
Crossover beetween [ 0  0 -1] and [ 1 -2  1] Resulted in [ 0.91712958 -1.83425917  0.83425917] and [ 0.08287042 -0.16574083 -0.83425917]


In [116]:
def mutate(chromosome):
    mutated = False
    for i in range(len(chromosome)):
        if random.uniform(0,1) <= mutation_probability:
            print(f"Performing mutation on chromosome {chromosome} at index {i}")
            mutated = True
            
    if mutated:
        print(f"Resulting chromosome: {chromosome}\n")
    else:
        print(f"No mutation for {chromosome}\n")
    
    return chromosome

    
    

SyntaxError: incomplete input (532791383.py, line 3)